In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### easy  
##### взять один файл с маркой машины
##### Построить прогноз, используя модели линейной регрессии и случайного леса. Сравнить результаты, используя кросс-валидацию. Подобрать гиперпараметры моделей (для линейной регрессии можно использовать реализацию с градиентным спуском SGDRegressor). Оценить качество, используя метрику r2_score. Оценить важность признаков (параметр feature_importance_ у случайного леса).

открытие файла с данными Ford

In [2]:
df_ford = pd.read_csv('avto/ford.csv').dropna()
df_ford.head()

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,Fiesta,2017,12000,Automatic,15944,Petrol,150,57.7,1.0
1,Focus,2018,14000,Manual,9083,Petrol,150,57.7,1.0
2,Focus,2017,13000,Manual,12456,Petrol,150,57.7,1.0
3,Fiesta,2019,17500,Manual,10460,Petrol,145,40.3,1.5
4,Fiesta,2019,16500,Automatic,1482,Petrol,145,48.7,1.0


определение x и y, обработка категориальных данных

In [3]:
x_ford = pd.concat([
    pd.get_dummies(df_ford['model']),
    df_ford['year'],
    pd.get_dummies(df_ford['transmission']), 
    df_ford['mileage'], 
    pd.get_dummies(df_ford['fuelType']), 
    df_ford['tax'], 
    df_ford['mpg'], 
    df_ford['engineSize'] ], axis=1)

x_ford.head()

,B-MAX,C-MAX,EcoSport,Edge,Escort,Fiesta,Focus,Fusion,Galaxy,Grand C-MAX,...,Semi-Auto,mileage,Diesel,Electric,Hybrid,Other,Petrol,tax,mpg,engineSize
0,0,0,0,0,0,1,0,0,0,0,...,0,15944,0,0,0,0,1,150,57.7,1.0
1,0,0,0,0,0,0,1,0,0,0,...,0,9083,0,0,0,0,1,150,57.7,1.0
2,0,0,0,0,0,0,1,0,0,0,...,0,12456,0,0,0,0,1,150,57.7,1.0
3,0,0,0,0,0,1,0,0,0,0,...,0,10460,0,0,0,0,1,145,40.3,1.5
4,0,0,0,0,0,1,0,0,0,0,...,0,1482,0,0,0,0,1,145,48.7,1.0


In [4]:
y_ford = df_ford[['price']]
y_ford.head()

,price
0,12000
1,14000
2,13000
3,17500
4,16500


при определении y_ford = y_ford.values нормально работает LinearRegression(), а RandomForestClassifier() и SGDRegressor() выдают бесконечное количество предупреждений: "DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel()."  
при определении y_ford = y_ford.values.ravel() кривые результаты начинает выдавать LinearRegression()  
поэтому будет две части: одна с y_ford = y_ford.values для LinearRegression() и вторая с y_ford = y_ford.values.ravel()

модель линейной регрессии

In [5]:
y_ford = df_ford[['price']].values

разбивка на train и test (80% - 20%)

In [6]:
x_train_ford, x_test_ford, y_train_ford, y_test_ford = train_test_split(x_ford, y_ford, train_size=0.8)

In [7]:
model_lr = LinearRegression()
model_lr.fit(x_train_ford, y_train_ford)
y_pred_lr = model_lr.predict(x_test_ford)
r2_score(y_pred_lr, y_test_ford)

-0.00027805465603503166

In [8]:
cr_val_lr = cross_val_score(model_lr, x_train_ford, y_train_ford)
print('ЛИНЕЙНАЯ РЕГРЕССИЯ')
print('кросс-валидация на train выборке:',cr_val_lr)
print('cреднее по кросс-валидации:', cr_val_lr.mean())
print('качество на тестовой выборке:', r2_score(y_pred_lr, y_test_ford))

ЛИНЕЙНАЯ РЕГРЕССИЯ
кросс-валидация на train выборке: [-6.29009642e+11  7.93963251e-01  8.57533525e-01 -4.42695718e+11
  8.46260571e-01]
cреднее по кросс-валидации: -214341072031.53525
качество на тестовой выборке: -0.00027805465603503166


модель случайного леса (без указания max_depth вылетает - не хватает памяти)

In [9]:
y_ford = df_ford[['price']].values.ravel()

In [10]:
x_train_ford, x_test_ford, y_train_ford, y_test_ford = train_test_split(x_ford, y_ford, train_size=0.8)

In [11]:
model_rfc = RandomForestClassifier(max_depth=5)
model_rfc.fit(x_train_ford, y_train_ford)
y_pred_rfc = model_rfc.predict(x_test_ford)
r2_score(y_pred_rfc, y_test_ford)

0.6834828924271061

In [12]:
r2=r2_score(y_pred_rfc, y_test_ford)
cr_val_rfc = cross_val_score(model_rfc, x_train_ford, y_train_ford, error_score=r2)
print('СЛУЧАЙНЫЙ ЛЕС')
print('кросс-валидация на train выборке:',cr_val_rfc)
print('cреднее по кросс-валидации:', cr_val_rfc.mean())
print('качество на тестовой выборке:', r2_score(y_pred_rfc, y_test_ford))

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


СЛУЧАЙНЫЙ ЛЕС
кросс-валидация на train выборке: [0.02191304 0.02991304 0.02540014 0.0302714  0.02574809]
cреднее по кросс-валидации: 0.02664914223472815
качество на тестовой выборке: 0.6834828924271061


SGDRegressor

In [13]:
model_sgdr = SGDRegressor()
model_sgdr.fit(x_train_ford, y_train_ford)
y_pred_sgdr = model_sgdr.predict(x_test_ford)
r2_score(y_pred_sgdr, y_test_ford)

-1.1955888327866329

подбор гиперпараметров

In [14]:
params = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.0005, 0.1],
    'max_iter': [100, 500, 1000]
}

sgdr_search = RandomizedSearchCV(SGDRegressor(), params)
sgdr_search.fit(x_train_ford, y_train_ford)
sgdr_search.best_estimator_

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


SGDRegressor(alpha=0.1, max_iter=100, penalty='l1')

In [15]:
#!!!!!
params = {
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200],
}

rfc_search = RandomizedSearchCV(RandomForestClassifier(), params)
rfc_search.fit(x_train_ford, y_train_ford)
rfc_search.best_estimator_

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomForestClassifier(max_depth=5)

оценка важности признаков

In [16]:
model_rfc.feature_importances_

array([0.00407354, 0.00874137, 0.00797129, 0.0036452 , 0.00066525,
       0.03348074, 0.01814589, 0.00142533, 0.00638403, 0.00607111,
       0.00479408, 0.00646057, 0.02029018, 0.03046129, 0.02447486,
       0.00181632, 0.00584318, 0.00035153, 0.0057274 , 0.00032693,
       0.00296698, 0.00414156, 0.        , 0.17241115, 0.02122525,
       0.02434794, 0.01628875, 0.18533113, 0.02898467, 0.00137282,
       0.00458447, 0.00151808, 0.02949934, 0.1032947 , 0.11371314,
       0.09916995])

In [17]:
print('наибольшая значимость признака: ', model_rfc.feature_importances_.max())
print('номер столбца с наиболее значимым признаком', np.argmax(model_rfc.feature_importances_))

наибольшая значимость признака:  0.18533112592016085
номер столбца с наиболее значимым признаком 27


In [18]:
x_ford = pd.concat([
    pd.get_dummies(df_ford['model']),
    df_ford['year'],
    pd.get_dummies(df_ford['transmission']), 
    df_ford['mileage'], 
    pd.get_dummies(df_ford['fuelType']), 
    df_ford['tax'], 
    df_ford['mpg'], 
    df_ford['engineSize'] ], axis=1)

x_ford.iloc[:0,23]

Series([], Name: year, dtype: int64)

получается, самый важный признак - **год выпуска**

### normal  
##### объединить в один датафрейм данные по всем маркам машин; преобразовать категориальные признаки
##### построить еще несколько моделей, используя подбор гиперпараметров; сравнить между собой все построенные модели

объединение в один датафрейм данных по всем маркам машин (кроме unclean cclass и unclean focus - насколько понимаю, это предварительные версии уже существующих файлов)

In [19]:
import glob
import os

In [20]:
mycsvdir = 'avto'
csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))

dataframes = []
for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    dataframes.append(df)

df_all = pd.concat(dataframes, ignore_index=True)
df_all

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,tax(£)
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4,NaN
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0,NaN
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4,NaN
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0,NaN
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...
108535,Eos,2012,5990,Manual,74000,Diesel,125.0,58.9,2.0,NaN
108536,Fox,2008,1799,Manual,88102,Petrol,145.0,46.3,1.2,NaN
108537,Fox,2009,1590,Manual,70000,Petrol,200.0,42.0,1.4,NaN
108538,Fox,2006,1250,Manual,82704,Petrol,150.0,46.3,1.2,NaN


удаление строк с пропущенными данными, преобразование категориальных признаков; столбец "tax(£)" пустой, можно удалить; определение x и y

In [21]:
df_all = df_all.drop(columns=['tax(£)']).dropna()
df_all

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize
0,A1,2017,12500,Manual,15735,Petrol,150.0,55.4,1.4
1,A6,2016,16500,Automatic,36203,Diesel,20.0,64.2,2.0
2,A1,2016,11000,Manual,29946,Petrol,30.0,55.4,1.4
3,A4,2017,16800,Automatic,25952,Diesel,145.0,67.3,2.0
4,A3,2019,17300,Manual,1998,Petrol,145.0,49.6,1.0
...,...,...,...,...,...,...,...,...,...
108535,Eos,2012,5990,Manual,74000,Diesel,125.0,58.9,2.0
108536,Fox,2008,1799,Manual,88102,Petrol,145.0,46.3,1.2
108537,Fox,2009,1590,Manual,70000,Petrol,200.0,42.0,1.4
108538,Fox,2006,1250,Manual,82704,Petrol,150.0,46.3,1.2


In [22]:
x_all = pd.concat([
    pd.get_dummies(df_all['model']),
    df_all['year'],
    pd.get_dummies(df_all['transmission']), 
    df_all['mileage'], 
    pd.get_dummies(df_all['fuelType']), 
    df_all['tax'], 
    df_all['mpg'], 
    df_all['engineSize'] ], axis=1)
x_all.head()

,1 Series,2 Series,3 Series,4 Series,5 Series,6 Series,7 Series,8 Series,A Class,A1,...,Semi-Auto,mileage,Diesel,Electric,Hybrid,Other,Petrol,tax,mpg,engineSize
0,0,0,0,0,0,0,0,0,0,1,...,0,15735,0,0,0,0,1,150.0,55.4,1.4
1,0,0,0,0,0,0,0,0,0,0,...,0,36203,1,0,0,0,0,20.0,64.2,2.0
2,0,0,0,0,0,0,0,0,0,1,...,0,29946,0,0,0,0,1,30.0,55.4,1.4
3,0,0,0,0,0,0,0,0,0,0,...,0,25952,1,0,0,0,0,145.0,67.3,2.0
4,0,0,0,0,0,0,0,0,0,0,...,0,1998,0,0,0,0,1,145.0,49.6,1.0


In [23]:
x_all = x_all.values
y_all = df_all[['price']].values

In [24]:
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(x_all, y_all, train_size=0.8)

**модель случайного леса**

In [25]:
model_rfc_a = RandomForestClassifier(max_depth=5)
model_rfc_a.fit(x_train_all, y_train_all)
y_pred_rfc_a = model_rfc_a.predict(x_test_all)
r2_score(y_pred_rfc_a, y_test_all)

C:\Users\79216\AppData\Local\Temp/ipykernel_16060/3213653613.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rfc_a.fit(x_train_all, y_train_all)


-0.0922143874532928

кросс-валидация

In [33]:
#!!!!!!!
r2=r2_score(y_pred_rfc_a, y_test_all)
cr_val_rfc_a = cross_val_score(model_rfc_a, x_train_all, y_train_all, error_score=r2)
print('кросс-валидация на train выборке:',cr_val_rfc_a)
print('cреднее по кросс-валидации:', cr_val_rfc_a.mean())

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_validation.py:681: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the

кросс-валидация на train выборке: [0.01749155 0.01510734 0.01676385 0.01557116 0.01749271]
cреднее по кросс-валидации: 0.016485323462067648


подбор гиперпараметров

In [34]:
#!!!!!!
params = {
    'max_depth': [3, 4, 5],
    'n_estimators': [50, 100, 200],
    'criterion': ['gini','entropy']
}

rfc_search = RandomizedSearchCV(RandomForestClassifier(), params)
rfc_search.fit(x_train_ford, y_train_ford)
rfc_search.best_estimator_

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


RandomForestClassifier(criterion='entropy', max_depth=5)

In [35]:
model_rfc_a = RandomForestClassifier(criterion='entropy', max_depth=5, n_estimators=50)
model_rfc_a.fit(x_train_all, y_train_all)
y_pred_rfc_a = model_rfc_a.predict(x_test_all)
r2_score(y_pred_rfc_a, y_test_all)

C:\Users\79216\AppData\Local\Temp/ipykernel_16060/425115558.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_rfc_a.fit(x_train_all, y_train_all)


-0.08794211278045272

**модель SGDRegressor**

In [36]:
model_sgdr_a = SGDRegressor()
model_sgdr_a.fit(x_train_all, y_train_all)
y_pred_sgdr_a = model_sgdr_a.predict(x_test_all)
r2_score(y_pred_sgdr_a, y_test_all)

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-1.245233988623926

кросс-валидация

In [37]:
r2=r2_score(y_pred_sgdr_a, y_test_all)
cr_val_sgdr_a = cross_val_score(model_sgdr_a, x_train_all, y_train_all, error_score=r2)
print('кросс-валидация на train выборке:',cr_val_sgdr_a)
print('cреднее по кросс-валидации:', cr_val_sgdr_a.mean())

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please 

кросс-валидация на train выборке: [-2.17703715e+25 -3.41989759e+26 -8.44153936e+25 -2.12052676e+26
 -3.03068341e+27]
cреднее по кросс-валидации: -7.381823222377306e+26


подбор гиперпараметров

In [38]:
params = {
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.0005, 0.1],
    'max_iter': [50, 100, 200]
}

sgdr_a_search = RandomizedSearchCV(SGDRegressor(), params)
sgdr_a_search.fit(x_train_ford, y_train_ford)
sgdr_a_search.best_estimator_

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:1503: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(
C:\Users\79216\a

SGDRegressor(alpha=0.1, max_iter=200)

In [39]:
model_sgdr_a = SGDRegressor(alpha=0.0005, max_iter=200, penalty='elasticnet')
model_sgdr_a.fit(x_train_all, y_train_all)
y_pred_sgdr_a = model_sgdr_a.predict(x_test_all)
r2_score(y_pred_sgdr_a, y_test_all)

C:\Users\79216\anaconda3\envs\py38\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


-0.785952886518696

**лучше результаты у SGDRegressor**